Import Models

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader

In [10]:
from torch.utils.data import Dataset, Subset, sampler, DataLoader
import torch
import pandas as pd
import numpy as np

validation_split = 0.2
shuffle_dataset = True
random_seed = 44
batch_size = 16384

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    print("no gpu")


# read in data
class RideHailingDataset(Dataset):
    def __init__(self, csv_file, label_file):
        self.csv_data = torch.Tensor(pd.read_csv(csv_file).to_numpy()[:, 1:]).to(device)
        self.label_data = torch.Tensor((pd.read_csv(label_file)).to_numpy()[:, 1]).view(-1).to(device)
        print(self.csv_data.size())
        print(self.label_data.size())
    def __len__(self):
        return len(self.csv_data)
    def __getitem__(self, idx):
        data = (self.csv_data[idx], self.label_data[idx])
        return data


dataset = RideHailingDataset("data.csv", "label.csv")
dataset_size = len(dataset)
print(dataset[0])

indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = sampler.SubsetRandomSampler(train_indices)
valid_sampler = sampler.SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

torch.Size([2847606, 25])
torch.Size([2847606])
(tensor([-0.8755,  1.0000,  0.0000,  0.5705,  0.9098,  0.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4277, -0.5651,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         0.0000], device='cuda:0'), tensor(0.5649, device='cuda:0'))


Setting Parameters

Create Model

In [3]:
class model(nn.Module):
    def __init__(self,input_size=25,hidden_size=50,output_size=1):
        super(model, self).__init__()
        self.features = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size,hidden_size),
        )
        
    def forward(self,x):
        x = self.features(x)
        x = torch.sum(x, dim=1)
        return x
    
model = model(25,50,1).cuda()

In [4]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3*1e-4)

Training

In [ ]:
model.train()

In [155]:

for e in range(1):
    lossE = 0
    count = 0
    for batch_index, (X, Y) in enumerate(train_loader):
        #forward
        #print(X.size())
        optimizer.zero_grad()
        out = model(X)
        

        loss = criterion(out,Y)
        #backward
        lossE += loss.data.item()
        loss.backward()
        count += 1
        optimizer.step()
    print('Epoch: {}, Loss: {:.5f}'.format(e + 1, lossE / count))
          

Epoch: 1, Loss: 0.39655
Epoch: 2, Loss: 0.39758
Epoch: 3, Loss: 0.39686


KeyboardInterrupt: 

Prediction

In [18]:
model = model.eval() #turn into test model

errorB = 0
accB = 0
rB = 0
count = 0
 #prediction result
for batch_index, (X, Y) in enumerate(validation_loader):
    out = model(X).view(-1)
    print(out[0])
    print(X[0])
    print(Y.t()[0])
    error = sum(abs(out - Y.t())).item() / batch_size
    acc = sum(abs(out - Y.t()) < 0.5).item() / batch_size
    #print("loss ", error)
    errorB += error
    #print("acc ", acc)
    accB += acc

    _out = out.cpu().detach().numpy()
    _Y = Y.t().cpu().detach().numpy()
    out_Mean = np.mean(_out)
    Y_Mean = np.mean(_Y)
    outStd = np.std(_out)
    YStd = np.std(_Y)
    Z_out = (_out - out_Mean) / outStd
    Z_Y = (_Y - Y_Mean) / YStd
    r = np.sum(Z_out * Z_Y) / batch_size
    rB += r
    #print("relation ", r)
    count += 1
print("loss ", errorB / count)
print("acc ", accB / count)
print("r ", rB / count) 
    
    

tensor(0.6538, device='cuda:0', grad_fn=<SelectBackward>)
tensor([-0.8955,  0.0000,  1.0000, -0.3240, -0.0721,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0014, -2.3579,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000], device='cuda:0')
tensor(-0.2366, device='cuda:0')
tensor(-0.4539, device='cuda:0', grad_fn=<SelectBackward>)
tensor([-1.1166,  1.0000,  0.0000,  0.4976, -0.8901,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0038, -0.3919,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         1.0000], device='cuda:0')
tensor(-0.5243, device='cuda:0')
tensor(-0.0558, device='cuda:0', grad_fn=<SelectBackward>)
tensor([ 1.2636,  1.0000,  0.0000,  0.1752, -0.3837,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5713, -1.5716,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0

In [ ]:
# Save for c++ to use

In [173]:
sm = torch.jit.script(model.cpu())
torch.jit.save(sm, "model.pth")



In [17]:
model = torch.jit.load("model.pth").to(device)